## Import library

In [27]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os
import pandas.io.sql as sqlio
# import psycopg2
import calendar
# from query_postgre import *

## Read the excel file name to extract date

In [28]:
# file_name = os.listdir('Incentive_CAL')[0]
# month = file_name.split('_')[-2]
# year = file_name.split('_')[-1]

# end_date_of_month = calendar.monthrange(int(month), int(year))[1]

# start_date = f"{year}-{month}-01"
# end_date = f"{year}-{month}-{end_date_of_month}"

## Take NMV, SLOB from DB

In [29]:
# df_nmv = query_nmv()

In [30]:
def query_slob(start_date):
    with psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, port, dbname, user, password)) as conn:
        sql_nmv = f"""
        SELECT groupbrand, sum(NMV)
        FROM sales.v_sales_mtd_v2
        WHERE date(orderdate + interval '7 hours') between '{start_date}'::date AND '{end_date}'::date;
        """
        df_nmv = pd.read_sql_query(sql_nmv, conn)
        return df_nmv

In [31]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os

directory = "Incentive_CAL"
file_name = "COM incentive_Dec 2022_Duy_Anh.xlsx"
path = os.path.join(directory, file_name)
df = pd.read_excel(path, "Detail")
df = df.iloc[9:,1:]

new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

#take rows that contain specific 'Onboarded' in 'Store status'
df = df[df["Store status"] == "Onboarded"]

# Rename the cols
df.rename(columns={'Target NMV after': 'Target NMV','GP actual (Simulated)': 'GP actual', 'Target GP after (Simulated)': 'Target GP'}, inplace=True)

In [32]:
# Rule for calculating percentage
def per_cal(x,y):
    if y == 0 and x!=0:
        return 1
    elif y == 0 and x == 0:
        return 0
    elif y!=0:
        return x/y

In [33]:
# Create NMV/GP - B2B/B2C cols
func = np.vectorize(per_cal)

def extract_b_or_c(df):
    # Take only usefull cols
#     cols = ["KAM/KAE (Full name)", "Group brand", "Store", "B2B or B2C", "GBM/Team lead (Full name)", "CD (Full name)",
#             "NMV actual", 'Target NMV', 'GP actual', 'Target GP',
#             "SLOB performance", 'SLOB Before', 'SLOB After', 'SLOB']
#     df = df[cols]
    
    df['NMV actual B2B'] = np.where(df['B2B or B2C']== 'B2B', df['NMV actual'], 0)
    df['NMV target B2B'] = np.where(df['B2B or B2C']== 'B2B', df['Target NMV'], 0)
    df['% NMV B2B'] = np.where(df['B2B or B2C']== 'B2B', func(df['NMV actual'], df['Target NMV']), 0)

    df['NMV actual B2C'] = np.where(df['B2B or B2C']== 'B2C', df['NMV actual'], 0)
    df['NMV target B2C'] = np.where(df['B2B or B2C']== 'B2C', df['Target NMV'], 0)
    df['% NMV B2C'] = np.where(df['B2B or B2C']== 'B2C', func(df['NMV actual'], df['Target NMV']), 0)

    df['GP actual B2B'] = np.where(df['B2B or B2C']== 'B2B', df['GP actual'], 0)
    df['GP target B2B'] = np.where(df['B2B or B2C']== 'B2B', df['Target GP'], 0)
    df['% GP B2B'] = np.where(df['B2B or B2C']== 'B2B', func(df['GP actual'], df['Target GP']), 0)

    df['GP actual B2C'] = np.where(df['B2B or B2C']== 'B2C', df['GP actual'], 0)
    df['GP target B2C'] = np.where(df['B2B or B2C']== 'B2C', df['Target GP'], 0)
    df['% GP B2C'] = np.where(df['B2B or B2C']== 'B2C', func(df['GP actual'], df['Target GP']), 0)

    # Drop current NMV and GP after convert
    df = df.drop(['NMV actual', "Target NMV", "GP actual", "Target GP"], axis=1)
    rearange_cols =  ["KAM/KAE (Full name)", "Group brand", "Store", "B2B or B2C", "GBM/Team lead (Full name)", "CD (Full name)",
                      "NMV actual B2B", 'NMV target B2B', '% NMV B2B',
                      "NMV actual B2C", 'NMV target B2C', '% NMV B2C',
                      'GP actual B2B', 'GP target B2B', '% GP B2B',
                      'GP actual B2C', 'GP target B2C', '% GP B2C',
                      "SLOB performance", 'SLOB Before', 'SLOB After', 'SLOB']
    df = df[rearange_cols]
    return df

# Merge names
df['KAM/KAE (Full name)'] = df['KAM/KAE (Full name)'].str.title()
df['CD (Full name)'] = df['CD (Full name)'].str.title()
df['GBM/Team lead (Full name)'] = df['GBM/Team lead (Full name)'].str.title()

In [34]:
# Take list of GBMs
GBMs = df["GBM/Team lead (Full name)"].unique()

# Take list of All KAM under each GBM
KAM_under = []
for GBM in GBMs:
    GBM_df = df.loc[df["GBM/Team lead (Full name)"] == GBM]
    KAM_under.append(list(GBM_df["KAM/KAE (Full name)"].unique()))

In [35]:
# Func for sum of SLOB per groupbrand
# The SLOB of each channel of each groupbrand is the same. This function used to sum SLOB
# of 1 person without duplicate the SLOB value
def cal_sum_slob(df, col: str):
    gb = df["Group brand"].to_list()
    slob = df[col].to_list()
    s = 0
    hold = ""
    for i in range(len(gb)):
        if hold != gb[i]:
            s += slob[i]
            hold = gb[i]
    return s

# Convert df col to percentage
# Convert
def int_to_per(x:int):
    if x!= '':
        return "{:.2%}".format(float(x))

# Func for highlight_last_row
def highlight_last_row(df):
    color = 'white'
    if df.values[3] == 'KAM':
        color = 'yellow'
    if df.values[0] in GBMs:
        color = 'red'
    if df.values[0] in CDs:
        color = 'orange'

    return ['background-color: %s' % color]*len(df.values)

In [36]:
def take_detail_KAM(KAM_name: str):
    KAM_df = df[df["KAM/KAE (Full name)"] == KAM_name]
    KAM_df = extract_b_or_c(KAM_df)
    
    # Add Total row
    no_group_brand = KAM_df["Group brand"].nunique()
    no_store = KAM_df["Store"].nunique()
    no_GBM = KAM_df["GBM/Team lead (Full name)"].nunique()
    no_CD = KAM_df["CD (Full name)"].nunique()
    
    sum_nmv_actual_b2b = KAM_df["NMV actual B2B"].sum()
    sum_nmv_actual_b2c = KAM_df["NMV actual B2C"].sum()
    sum_nmv_target_b2b = KAM_df["NMV target B2B"].sum()
    sum_nmv_target_b2c = KAM_df["NMV target B2C"].sum()
    
    nmv_per_b2b = per_cal(sum_nmv_actual_b2b, sum_nmv_target_b2b)
    nmv_per_b2c = per_cal(sum_nmv_actual_b2c, sum_nmv_target_b2c)
    
    sum_gp_actual_b2b = KAM_df["GP actual B2B"].sum()
    sum_gp_actual_b2c = KAM_df["GP actual B2C"].sum()
    sum_gp_target_b2b = KAM_df["GP target B2B"].sum()
    sum_gp_target_b2c = KAM_df["GP target B2C"].sum()
    
    gp_per_b2b = per_cal(sum_gp_actual_b2b, sum_gp_target_b2b)
    gp_per_b2c = per_cal(sum_gp_actual_b2c, sum_gp_target_b2c) 
    
    sum_slob_before = cal_sum_slob(KAM_df, "SLOB Before")
    sum_slob_after = cal_sum_slob(KAM_df, "SLOB After")
    sum_slob = cal_sum_slob(KAM_df, "SLOB")
    slop_per = per_cal(sum_slob, sum_slob_before)

    KAM_df.loc[len(KAM_df.index)] = [KAM_name, no_group_brand, no_store, "KAM", no_GBM, no_CD,
                                     sum_nmv_actual_b2b, sum_nmv_target_b2b, nmv_per_b2b,
                                     sum_nmv_actual_b2c, sum_nmv_target_b2c, nmv_per_b2c,
                                     sum_gp_actual_b2b, sum_gp_target_b2b, gp_per_b2b,
                                     sum_gp_actual_b2c, sum_gp_target_b2c, gp_per_b2c,
                                     slop_per, sum_slob_before, sum_slob_after, sum_slob]
    
    # Change to percentage
    conv_to_per = ["% NMV B2B", "% NMV B2C", "% GP B2B","% GP B2C", "SLOB performance"]
    for col_name in conv_to_per:
        KAM_df[col_name] = KAM_df[col_name].apply(int_to_per)
        
    # We dont send the sum_slob_before, sum_slob_after, sum_slob
    KAM_df = KAM_df.drop(["SLOB Before", "SLOB After", "SLOB"], axis = 1)
    return KAM_df  

# Define the threshold and Incentive

In [37]:
# Define the threshold of calculating the incentive
NMV_threshold = [0.85, 1, 1.25, 1.5]
GP_threshold = [0.85, 1, 1.25, 1.5]
SLOB_threshold = [-0.15]

# Define the incentive based on threshold
NMV_incentive_values = [0.02, 0.08, 0.12, 0.2]
NMV_incentive_B2B_values = [0.004, 0.016, 0.024, 0.04]
NMV_incentive_B2C_values = [0.016, 0.064, 0.096, 0.16]

GP_incentive_values = [0.03, 0.12, 0.18, 0.3]
GP_incentive_B2B_values = [0.006, 0.024, 0.036, 0.06]
GP_incentive_B2C_values = [0.024, 0.096, 0.144, 0.24]

SLOB_incentive_values = [0.05]

## Function Calculate the incentive

In [38]:
def cal_incentive(role: str):    
    # Take all name of role
    if role == "KAM":
        role_col = "KAM/KAE (Full name)"
        NMV_target_col = "Target NMV"
        GP_target_col = "Target GP"
        
    elif role == "GBM":
        role_col = "GBM/Team lead (Full name)"
        NMV_target_col = "Target NMV"
        GP_target_col = "Target GP"
        
    elif role == "CD":
        role_col = "CD (Full name)"
        NMV_target_col = "Target NMV"
        GP_target_col = "Target GP"
    
    people = list(df[role_col].unique())
   
    # Calculate the incentive of each person
    people_incentive = []
    
    people_total_NMV_actual_B2B = []
    people_total_NMV_target_B2B = []
    people_percentage_NMV_B2B = []
    
    people_total_GP_actual_B2B = []
    people_total_GP_target_B2B = []
    people_percentage_GP_B2B = []
    
    people_total_NMV_actual_B2C = []
    people_total_NMV_target_B2C = []
    people_percentage_NMV_B2C = []
    
    people_total_GP_actual_B2C = []
    people_total_GP_target_B2C = []
    people_percentage_GP_B2C = []
    
    people_total_SLOB_performance = []
    
    for person in people:
        person_df = df.loc[df[role_col] == person]
        # only B2C
        if "B2B" not in person_df["B2B or B2C"].unique():
            # Cal incentive of NMV
            person_NMV_actual = person_df["NMV actual"].sum()
            person_NMV_target = person_df[NMV_target_col].sum()
            person_percentage_NMV = per_cal(person_NMV_actual,person_NMV_target)
            
            if person_percentage_NMV >= NMV_threshold[0] and person_percentage_NMV < NMV_threshold[1]:
                person_incentive_NMV = NMV_incentive_values[0]
            elif person_percentage_NMV >= NMV_threshold[1] and person_percentage_NMV < NMV_threshold[2]:
                person_incentive_NMV = NMV_incentive_values[1]
            elif person_percentage_NMV >= NMV_threshold[2] and person_percentage_NMV < NMV_threshold[3]:
                person_incentive_NMV = NMV_incentive_values[2]
            elif person_percentage_NMV >= NMV_threshold[3]:
                person_incentive_NMV = NMV_incentive_values[3]
            else:
                person_incentive_NMV = 0

            # Cal incentive of GP
            person_GP_actual = person_df["GP actual"].sum()
            person_GP_target = person_df[GP_target_col].sum()
            person_percentage_GP = per_cal(person_GP_actual,person_GP_target)
            
            if person_percentage_GP >= GP_threshold[0] and person_percentage_GP < GP_threshold[1]:
                person_incentive_GP = GP_incentive_values[0]
            elif person_percentage_GP >= GP_threshold[1] and person_percentage_GP < GP_threshold[2]:
                person_incentive_GP = GP_incentive_values[1]
            elif person_percentage_GP >= GP_threshold[2] and person_percentage_GP < GP_threshold[3]:
                person_incentive_GP = GP_incentive_values[2]
            elif person_percentage_GP >= GP_threshold[3]:
                person_incentive_GP = GP_incentive_values[3]
            else:
                person_incentive_GP = 0
                
            # Add to list
            people_total_NMV_actual_B2C.append(person_NMV_actual)
            people_total_NMV_target_B2C.append(person_NMV_target)
            people_percentage_NMV_B2C.append(person_percentage_NMV)
            
            people_total_GP_actual_B2C.append(person_GP_actual)
            people_total_GP_target_B2C.append(person_GP_target)
            people_percentage_GP_B2C.append(person_percentage_GP)
            
            people_total_NMV_actual_B2B.append("")
            people_total_NMV_target_B2B.append("")
            people_percentage_NMV_B2B.append("")
            
            people_total_GP_actual_B2B.append("")
            people_total_GP_target_B2B.append("")
            people_percentage_GP_B2B.append("")

        # B2B mixed with B2C
        else:
            person_df_B2B = person_df.loc[df["B2B or B2C"] == "B2B"]
            person_df_B2C = person_df.loc[df["B2B or B2C"] == "B2C"]

            # Cal incentive of NMV
            # B2B
            person_NMV_actual_B2B = person_df_B2B["NMV actual"].sum()
            person_NMV_target_B2B = person_df_B2B[NMV_target_col].sum()
            person_percentage_NMV_B2B = per_cal(person_NMV_actual_B2B, person_NMV_target_B2B)

            #
            if person_percentage_NMV_B2B >= NMV_threshold[0] and person_percentage_NMV_B2B < NMV_threshold[1]:
                person_incentive_NMV_B2B = NMV_incentive_B2B_values[0]
            elif person_percentage_NMV_B2B >= NMV_threshold[1] and person_percentage_NMV_B2B < NMV_threshold[2]:
                person_incentive_NMV_B2B = NMV_incentive_B2B_values[1]
            elif person_percentage_NMV_B2B >= NMV_threshold[2] and person_percentage_NMV_B2B < NMV_threshold[3]:
                person_incentive_NMV_B2B = NMV_incentive_B2B_values[2]
            elif person_percentage_NMV_B2B >= NMV_threshold[3]:
                person_incentive_NMV_B2B = NMV_incentive_B2B_values[3]
            else:
                person_incentive_NMV_B2B = 0
        
            # B2C
            person_NMV_actual_B2C = person_df_B2C["NMV actual"].sum()
            person_NMV_target_B2C = person_df_B2C[NMV_target_col].sum()
            person_percentage_NMV_B2C = per_cal(person_NMV_actual_B2C, person_NMV_target_B2C)
            
            #
            if person_percentage_NMV_B2C >= NMV_threshold[0] and person_percentage_NMV_B2C < NMV_threshold[1]:
                person_incentive_NMV_B2C = NMV_incentive_B2C_values[0]
            elif person_percentage_NMV_B2C >= NMV_threshold[1] and person_percentage_NMV_B2C < NMV_threshold[2]:
                person_incentive_NMV_B2C = NMV_incentive_B2C_values[1]
            elif person_percentage_NMV_B2C >= NMV_threshold[2] and person_percentage_NMV_B2C < NMV_threshold[3]:
                person_incentive_NMV_B2C = NMV_incentive_B2C_values[2]
            elif person_percentage_NMV_B2C >= NMV_threshold[3]:
                person_incentive_NMV_B2C = NMV_incentive_B2C_values[3]
            else:
                person_incentive_NMV_B2C = 0

            person_incentive_NMV = person_incentive_NMV_B2B + person_incentive_NMV_B2C

            # Cal incentive of GP
            # B2B
            person_GP_actual_B2B = person_df_B2B["GP actual"].sum()
            person_GP_target_B2B = person_df_B2B[GP_target_col].sum()
            person_percentage_GP_B2B = per_cal(person_GP_actual_B2B,person_GP_target_B2B)

            #
            if person_percentage_GP_B2B >= GP_threshold[0] and person_percentage_GP_B2B < GP_threshold[1]:
                person_incentive_GP_B2B = GP_incentive_B2B_values[0]
            elif person_percentage_GP_B2B >= GP_threshold[1] and person_percentage_GP_B2B < GP_threshold[2]:
                person_incentive_GP_B2B = GP_incentive_B2B_values[1]
            elif person_percentage_GP_B2B >= GP_threshold[2] and person_percentage_GP_B2B < GP_threshold[3]:
                person_incentive_GP_B2B = GP_incentive_B2B_values[2]
            elif person_percentage_GP_B2B >= GP_threshold[3]:
                person_incentive_GP_B2B = GP_incentive_B2B_values[3]
            else:
                person_incentive_GP_B2B = 0

            # B2C
            person_GP_actual_B2C = person_df_B2C["GP actual"].sum()
            person_GP_target_B2C = person_df_B2C[GP_target_col].sum()
            person_percentage_GP_B2C = per_cal(person_GP_actual_B2C,person_GP_target_B2C)

            if person_percentage_GP_B2C >= GP_threshold[0] and person_percentage_GP_B2C < GP_threshold[1]:
                person_incentive_GP_B2C = GP_incentive_B2C_values[0]
            elif person_percentage_GP_B2C >= GP_threshold[1] and person_percentage_GP_B2C < GP_threshold[2]:
                person_incentive_GP_B2C = GP_incentive_B2C_values[1]
            elif person_percentage_GP_B2C >= GP_threshold[2] and person_percentage_GP_B2C < GP_threshold[3]:
                person_incentive_GP_B2C = GP_incentive_B2C_values[2]
            elif person_percentage_GP_B2C >= GP_threshold[3]:
                person_incentive_GP_B2C = GP_incentive_B2C_values[3]
            else:
                person_incentive_GP_B2C = 0
            
            person_incentive_GP = person_incentive_GP_B2B + person_incentive_GP_B2C

            
            # Add to list
            people_total_NMV_actual_B2C.append(person_NMV_actual_B2C)
            people_total_NMV_target_B2C.append(person_NMV_target_B2C)
            
            people_total_GP_actual_B2C.append(person_GP_actual_B2C)
            people_total_GP_target_B2C.append(person_GP_target_B2C)
            
            people_total_NMV_actual_B2B.append(person_NMV_actual_B2B)
            people_total_NMV_target_B2B.append(person_NMV_target_B2B)
            
            people_total_GP_actual_B2B.append(person_GP_actual_B2B)
            people_total_GP_target_B2B.append(person_GP_target_B2B)
            
            people_percentage_NMV_B2C.append(person_percentage_NMV_B2C)
            people_percentage_GP_B2C.append(person_percentage_GP_B2C)
            people_percentage_NMV_B2B.append(person_percentage_NMV_B2B)
            people_percentage_GP_B2B.append(person_percentage_GP_B2B)
            
        # Cal incentive of SLOB
        if person_df["SLOB Before"].sum() == 0:
            person_incentive_SLOB = 0
            person_percentage_SLOB = 0
        else:
            sum_SLOB = cal_sum_slob(person_df, 'SLOB')
            sum_SLOB_Before = cal_sum_slob(person_df, 'SLOB Before')
            person_percentage_SLOB = sum_SLOB / sum_SLOB_Before
            
            if person_percentage_SLOB <= SLOB_threshold[0]:
                person_incentive_SLOB = SLOB_incentive_values[0]
            else:
                    person_incentive_SLOB = 0

        # Sum all incentive of KAM
        person_incentive = person_incentive_NMV + person_incentive_GP + person_incentive_SLOB

        # Add to list
        people_incentive.append(person_incentive)
        people_total_SLOB_performance.append(person_percentage_SLOB)

    return people, people_total_NMV_actual_B2B, people_total_NMV_target_B2B, people_percentage_NMV_B2B,people_total_GP_actual_B2B, people_total_GP_target_B2B, people_percentage_GP_B2B, people_total_NMV_actual_B2C, people_total_NMV_target_B2C, people_percentage_NMV_B2C, people_total_GP_actual_B2C, people_total_GP_target_B2C, people_percentage_GP_B2C, people_total_SLOB_performance, people_incentive

### Create the role_df

In [39]:
KAMs, KAM_total_NMV_actual_B2B, KAM_total_NMV_target_B2B, KAM_percentage_NMV_B2B, KAM_total_GP_actual_B2B, KAM_total_GP_target_B2B, KAM_percentage_GP_B2B,KAM_total_NMV_actual_B2C, KAM_total_NMV_target_B2C, KAM_percentage_NMV_B2C, KAM_total_GP_actual_B2C, KAM_total_GP_target_B2C, KAM_percentage_GP_B2C, KAM_total_SLOB_performance, KAM_incentive  = cal_incentive("KAM")
gbm_list, GBM_total_NMV_actual_B2B, GBM_total_NMV_target_B2B, GBM_percentage_NMV_B2B, GBM_total_GP_actual_B2B, GBM_total_GP_target_B2B, GBM_percentage_GP_B2B,GBM_total_NMV_actual_B2C, GBM_total_NMV_target_B2C, GBM_percentage_NMV_B2C, GBM_total_GP_actual_B2C, GBM_total_GP_target_B2C, GBM_percentage_GP_B2C, GBM_total_SLOB_performance, GBM_incentive  = cal_incentive("GBM")
cd_list, CD_total_NMV_actual_B2B, CD_total_NMV_target_B2B, CD_percentage_NMV_B2B, CD_total_GP_actual_B2B, CD_total_GP_target_B2B, CD_percentage_GP_B2B,CD_total_NMV_actual_B2C, CD_total_NMV_target_B2C, CD_percentage_NMV_B2C, CD_total_GP_actual_B2C, CD_total_GP_target_B2C, CD_percentage_GP_B2C, CD_total_SLOB_performance, CD_incentive  = cal_incentive("CD")

excel_df_cols = ["Name", "NMV actual B2B", "NMV target B2B", "% NMV B2B","NMV actual B2C", "NMV target B2C","% NMV B2C", "GP actual B2B", "GP target B2B", "% GP B2B","GP actual B2C", "GP target B2C", "% GP B2C","SLOB performance", "Incentive"]

In [40]:
KAMs_df = pd.DataFrame(list(zip(KAMs, KAM_total_NMV_actual_B2B, KAM_total_NMV_target_B2B, KAM_percentage_NMV_B2B, KAM_total_NMV_actual_B2C, KAM_total_NMV_target_B2C, KAM_percentage_NMV_B2C, KAM_total_GP_actual_B2B, KAM_total_GP_target_B2B, KAM_percentage_GP_B2B, KAM_total_GP_actual_B2C, KAM_total_GP_target_B2C, KAM_percentage_GP_B2C, KAM_total_SLOB_performance, KAM_incentive)),
                  columns = excel_df_cols)
conv_to_per = ["% NMV B2B", "% NMV B2C", "% GP B2B","% GP B2C", "SLOB performance", "Incentive"]
for col_name in conv_to_per:
    KAMs_df[col_name] = KAMs_df[col_name].apply(int_to_per)

In [41]:
GBMs_df = pd.DataFrame(list(zip(gbm_list, GBM_total_NMV_actual_B2B, GBM_total_NMV_target_B2B, GBM_percentage_NMV_B2B, GBM_total_NMV_actual_B2C, GBM_total_NMV_target_B2C, GBM_percentage_NMV_B2C, GBM_total_GP_actual_B2B, GBM_total_GP_target_B2B, GBM_percentage_GP_B2B, GBM_total_GP_actual_B2C, GBM_total_GP_target_B2C, GBM_percentage_GP_B2C, GBM_total_SLOB_performance, GBM_incentive)),
                  columns = excel_df_cols)
for col_name in conv_to_per:
    GBMs_df[col_name] = GBMs_df[col_name].apply(int_to_per)

In [42]:
CDs_df = pd.DataFrame(list(zip(cd_list, CD_total_NMV_actual_B2B, CD_total_NMV_target_B2B, CD_percentage_NMV_B2B, CD_total_NMV_actual_B2C, CD_total_NMV_target_B2C, CD_percentage_NMV_B2C, CD_total_GP_actual_B2B, CD_total_GP_target_B2B, CD_percentage_GP_B2B, CD_total_GP_actual_B2C, CD_total_GP_target_B2C, CD_percentage_GP_B2C, CD_total_SLOB_performance, CD_incentive)),
                  columns = excel_df_cols)
conv_to_per = ["% NMV B2B", "% NMV B2C", "% GP B2B","% GP B2C", "SLOB performance", "Incentive"]
for col_name in conv_to_per:
    CDs_df[col_name] = CDs_df[col_name].apply(int_to_per)

## Merge KAM df and export sheet to excel

In [43]:
# Take list of GBM
CDs = df["CD (Full name)"].unique()
GBM_under = []
for CD in CDs:
    CD_df = df.loc[df["CD (Full name)"] == CD]
    GBM_under.append(list(CD_df["GBM/Team lead (Full name)"].unique()))

In [44]:
if not os.path.exists('result'):
    os.mkdir('result')

for i in range(len(GBMs)):
    GBM = GBMs[i]
    writer = pd.ExcelWriter(os.path.join("result",f"{GBM}.xlsx"), engine = 'xlsxwriter')

    df_final = take_detail_KAM(KAM_under[i][0])
    for k in range(1,len(KAM_under[i])):
        df_current = take_detail_KAM(KAM_under[i][k])
        df_final = pd.concat([df_final, df_current])
    df_final = df_final.reset_index(drop=True)
    df_final = df_final.style.apply(highlight_last_row, axis=1)
    df_final.to_excel(writer, index = False, sheet_name = "Specific")
    
    ###################
    
    KAMs_under_GBMs = KAM_under[i]
    df_GBM = KAMs_df.loc[KAMs_df["Name"].isin(KAMs_under_GBMs)]
    df_GBM_lastrow = GBMs_df[GBMs_df["Name"] == GBM]
    df_GBM = pd.concat([df_GBM, df_GBM_lastrow])
        
    df_GBM = df_GBM.reset_index(drop=True)
    
    df_GBM = df_GBM.style.apply(highlight_last_row, axis=1)
        
    df_GBM.to_excel(writer, index = False, sheet_name = "General")

    writer.close()


# CD Excel

In [45]:
if not os.path.exists(os.path.join('result', 'CD')):
    os.mkdir(os.path.join('result', 'CD'))

for j in range(len(CDs)):
    CD = CDs[j]   
    writer = pd.ExcelWriter(f"result/CD/{CD}.xlsx", engine = 'xlsxwriter')
    for i in range(len(GBM_under[j])):
        GBM = GBM_under[j][i]
        index_of_GBM = list(GBMs).index(GBM)

        df_final = take_detail_KAM(KAM_under[index_of_GBM][0])
        for k in range(1,len(KAM_under[index_of_GBM])):
            df_current = take_detail_KAM(KAM_under[index_of_GBM][k])
            df_final = pd.concat([df_final, df_current])
        df_final = df_final.reset_index(drop=True)
        df_final = df_final.style.apply(highlight_last_row, axis=1)
        df_final.to_excel(writer, index = False, sheet_name = f"{GBM}-Specific")

        ###################

        KAMs_under_GBMs = KAM_under[index_of_GBM]
        df_GBM = KAMs_df.loc[KAMs_df["Name"].isin(KAMs_under_GBMs)]
        df_GBM_lastrow = GBMs_df[GBMs_df["Name"] == GBM]
        df_GBM = pd.concat([df_GBM, df_GBM_lastrow])

        df_GBM = df_GBM.reset_index(drop=True)

        df_GBM = df_GBM.style.apply(highlight_last_row, axis=1)

        df_GBM.to_excel(writer, index = False, sheet_name = f"{GBM}-General")
    
    GBMs_under_CDs = GBM_under[j]
    df_CD = GBMs_df.loc[GBMs_df["Name"].isin(GBMs_under_CDs)]
    df_CD_lastrow = CDs_df[CDs_df["Name"] == CD]
    df_CD = pd.concat([df_CD, df_CD_lastrow])
    
    df_CD = df_CD.reset_index(drop=True)

    df_CD = df_CD.style.apply(highlight_last_row, axis=1)

    df_CD.to_excel(writer, index = False, sheet_name = f"CD-{CD}")
    

    writer.close()    
    
    
# for j in range(len(CDs)):
#     CD = CDs[j]
    
#     writer = pd.ExcelWriter(f"result/CD/{CD}.xlsx", engine = 'xlsxwriter')
#     for i in range(len(GBM_under[j])):
#         GBM = GBM_under[j][i]
#         index_of_GBM = list(GBMs).index(GBM)

#         df_final = take_detail_KAM(KAM_under[index_of_GBM][0])
#         for k in range(1,len(KAM_under[index_of_GBM])):
#             df_current = take_detail_KAM(KAM_under[index_of_GBM][k])
#             df_final = pd.concat([df_final, df_current])
#         df_final = df_final.reset_index(drop=True)
#         df_final = df_final.style.apply(highlight_last_row, axis=1)
#         df_final.to_excel(writer, index = False, sheet_name = f"{GBM}-Specific")

#         ###################

#         KAMs_under_GBMs = KAM_under[index_of_GBM]
#         df_GBM = KAMs_df.loc[KAMs_df["Name"].isin(KAMs_under_GBMs)]
#         df_GBM_lastrow = GBMs_df[GBMs_df["Name"] == GBM]
#         df_GBM = pd.concat([df_GBM, df_GBM_lastrow])

#         df_GBM = df_GBM.reset_index(drop=True)

#         df_GBM = df_GBM.style.apply(highlight_last_row, axis=1)

#         df_GBM.to_excel(writer, index = False, sheet_name = f"{GBM}-General")
    
#     GBMs_under_CDs = GBM_under[j]
#     df_CD = GBMs_df.loc[GBMs_df["Name"].isin(GBMs_under_CDs)]
#     df_CD_lastrow = CDs_df[CDs_df["Name"] == CD]
#     df_CD = pd.concat([df_CD, df_CD_lastrow])
    
#     df_CD = df_CD.reset_index(drop=True)

#     df_CD = df_CD.style.apply(highlight_last_row, axis=1)

#     df_CD.to_excel(writer, index = False, sheet_name = f"CD-{CD}")
    

#     writer.close()


In [46]:
print('Done')

Done
